In [1]:
import requests #urllib,selenium
import urllib
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url = 'https://www.worldometers.info/world-population/population-by-country/'

In [3]:
data = requests.get(url)

In [4]:
data

<Response [200]>

# Task1

In [5]:
soup = BeautifulSoup(data.text)
columns = []
Country=[]
k=[]
Population = []
Yearly = []
NetChange = []
Density = []
LandArea = []
Migrants = []
FertRate = []
MedAge = []
UrbanPop = []
WorldShare = []

for i in soup.find_all('th'):
    columns.append(i.text)
    country_pop_df = pd.DataFrame(columns=columns)
for i in soup.find_all('td'):
    if i==None:
        continue
    else:
        k.append(i.text)
for i in soup.find_all('tr'):
    if i.a==None:
        continue
    else:
        Country.append(i.a.text)
country_pop_df['Country (or dependency)'] = Country

for z in range(11,0,-1):
    for i in range(len(k)):
        if (i+z)%12 == 0:
            if z == 10:
                Population.append(k[i])
            elif z == 9:
                Yearly.append(k[i])
            elif z == 8:
                NetChange.append(k[i])
            elif z == 7:
                Density.append(k[i])
            elif z == 6:
                LandArea.append(k[i])
            elif z == 5:
                Migrants.append(k[i])
            elif z == 4:
                FertRate.append(k[i])
            elif z == 3:
                MedAge.append(k[i])
            elif z == 2:
                UrbanPop.append(k[i])
            elif z == 1:
                WorldShare.append(k[i])
country_pop_df.drop(columns = '#',inplace=True)
country_pop_df['Population (2020)'] = Population
country_pop_df['Yearly Change'] = Yearly
country_pop_df['Net Change'] = NetChange
country_pop_df['Density (P/Km²)'] = Density
country_pop_df['Land Area (Km²)'] = LandArea
country_pop_df['Migrants (net)'] = Migrants
country_pop_df['Fert. Rate'] = FertRate
country_pop_df['Med. Age'] = MedAge
country_pop_df['Urban Pop %'] = UrbanPop
country_pop_df['World Share'] = WorldShare



In [6]:
country_pop_df.head()

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,"1,439,323,776",0.39 %,"5,540,090",153,"9,388,211","-348,399",1.7,38,61 %,18.47 %
1,India,"1,380,004,385",0.99 %,"13,586,631",464,"2,973,190","-532,687",2.2,28,35 %,17.70 %
2,United States,"331,002,651",0.59 %,"1,937,734",36,"9,147,420","954,806",1.8,38,83 %,4.25 %
3,Indonesia,"273,523,615",1.07 %,"2,898,047",151,"1,811,570","-98,955",2.3,30,56 %,3.51 %
4,Pakistan,"220,892,340",2.00 %,"4,327,022",287,"770,880","-233,379",3.6,23,35 %,2.83 %


## Task2

In [52]:
links =[]
soups_list = []
data_city = []
for i in soup.findAll('tr'):
    if i.a==None:
        continue
    else:
        href = i.a.get('href')
        links.append(href)
url = "https://www.worldometers.info/"
comp_links = [url+x for x in links]

for i in comp_links:
    data = requests.get(i)
    soups = BeautifulSoup(data.text)
    soups_list.append(soups)

for i in soups_list:
    for coun in i.find_all('div',class_ = 'content'):
        name = (coun.div.h1.text.split(' ')[1:-2])
        Country_Name = " ".join(name)  

    for j in i.findAll('table',class_='table table-hover table-condensed table-list'):            
            table_body = j.find('tbody')
            rows = table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                cols = cols + [Country_Name]
                data_city.append(cols)
City_df = pd.DataFrame(data_city,columns = ['S.NO','City','Population','Country'])
for coi in range(len(City_df)):
    City_df['Population'][coi] = City_df['Population'][coi].replace(',','')
City_df['Population'] = City_df['Population'].astype(str).astype(int)
idx = City_df.groupby(['Country'])['Population'].transform(max) == City_df['Population']
pop_cities_df = City_df[idx]
pop_cities_df.reset_index(drop =True,inplace=True)
pop_cities_df.drop(columns=['S.NO'],inplace=True)
pop_cities_df = pop_cities_df[['Country','City','Population']]
pop_cities_df.columns = ['Country','Most Populous City','Most Populous City Population']

C:\Users\saran borra\anaconda3\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [53]:
pop_cities_df.head()

,Country,Most Populous City,Most Populous City Population
0,China,Shanghai,22315474
1,India,Mumbai,12691836
2,United States,New York City,8175133
3,Indonesia,Jakarta,8540121
4,Pakistan,Karachi,11624219


## Task3

In [70]:
final_df = pd.merge(country_pop_df,pop_cities_df,how='left',left_on='Country (or dependency)',right_on='Country')
final_df = final_df.drop(columns=['Country'])
final_df['Most Populous City Population'] = final_df['Most Populous City Population'].astype('str')

In [71]:
final_df.head()

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share,Most Populous City,Most Populous City Population
0,China,"1,439,323,776",0.39 %,"5,540,090",153,"9,388,211","-348,399",1.7,38,61 %,18.47 %,Shanghai,22315474.0
1,India,"1,380,004,385",0.99 %,"13,586,631",464,"2,973,190","-532,687",2.2,28,35 %,17.70 %,Mumbai,12691836.0
2,United States,"331,002,651",0.59 %,"1,937,734",36,"9,147,420","954,806",1.8,38,83 %,4.25 %,New York City,8175133.0
3,Indonesia,"273,523,615",1.07 %,"2,898,047",151,"1,811,570","-98,955",2.3,30,56 %,3.51 %,Jakarta,8540121.0
4,Pakistan,"220,892,340",2.00 %,"4,327,022",287,"770,880","-233,379",3.6,23,35 %,2.83 %,Karachi,11624219.0


In [72]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235 entries, 0 to 234
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Country (or dependency)        235 non-null    object
 1   Population (2020)              235 non-null    object
 2   Yearly Change                  235 non-null    object
 3   Net Change                     235 non-null    object
 4   Density (P/Km²)                235 non-null    object
 5   Land Area (Km²)                235 non-null    object
 6   Migrants (net)                 235 non-null    object
 7   Fert. Rate                     235 non-null    object
 8   Med. Age                       235 non-null    object
 9   Urban Pop %                    235 non-null    object
 10  World Share                    235 non-null    object
 11  Most Populous City             195 non-null    object
 12  Most Populous City Population  235 non-null    object
dtypes: ob

## Task4

In [74]:
import pandas as pd

!wget -O ds_semi_processed.csv "https://drive.google.com/uc?id=1d1ub9_6iaWZ7yjgNLDRaUmQ8hWKrC39H"
#df = pd.read_csv('ds_semi_processed.csv')

'wget' is not recognized as an internal or external command,
operable program or batch file.
